In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
# fetch "New York City" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'New York City',
        'prop': 'extracts',
        # 'exintro': True,
        'explaintext': True,
    }
).json()
page = next(iter(response['query']['pages'].values()))
nyc_text = page['extract']

data_path = Path('data')
if not data_path.exists():
    Path.mkdir(data_path)

with open('data/nyc_text.txt', 'w') as fp:
    fp.write(nyc_text)

In [ ]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

In [4]:
from gpt_index import GPTTreeIndex, SimpleDirectoryReader

In [8]:
documents = SimpleDirectoryReader('data').load_data()
index = GPTTreeIndex.from_documents(documents)

INFO:gpt_index.indices.common.tree.base:> Building index from nodes: 0 chunks
> Building index from nodes: 0 chunks


AttributeError: 'GPTTreeIndexBuilder' object has no attribute '_llama_logger'

In [ ]:
index.save_to_disk('index.json')

In [ ]:
# try loading
new_index = GPTTreeIndex.load_from_disk('index.json')

In [ ]:
# GPT doesn't find the corresponding evidence in the leaf node, but still gives the correct answer
# set Logging to DEBUG for more detailed outputs

new_index.query("What is the name of the professional women's basketball team in New York City?")

In [ ]:
# GPT doesn't find the corresponding evidence in the leaf node, but still gives the correct answer
# set Logging to DEBUG for more detailed outputs

new_index.query("What battles took place in New York City in the American Revolution?")

In [ ]:
# GPT doesn't find the corresponding evidence in the leaf node, but still gives the correct answer
# set Logging to DEBUG for more detailed outputs

new_index.query("What are the airports in New York City?")

In [ ]:
# Try using embedding query
new_index.query("What are the airports in New York City?", mode="embedding")